Predicting total money spent on treating different cancers based on initial costs and patient demographics

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline


In [ ]:
df = pd.read_excel('DowloadableDataFull_2011.01.12.xlsx')
df.head(5)

In [ ]:
df.sample(10)

In [ ]:
df.isna().sum()

In [ ]:
df.info()

In [ ]:
df.shape()

In [ ]:
df.describe()

Cleaning

Exploring the Data

Plots

Feature Transformation and Engineering